In [2]:
from jupyter_plotly_dash import JupyterDash

import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import re
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "welcomeUser"
server = "localhost"
port = "54629"

shelter = AnimalShelter(username, password, server, port)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
limit_breeds = True

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H4('by Cory Remick 12/5/2022'))),
    html.Center(html.A(href='https://www.snhu.edu',
           children=html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),            
               alt="Grazioso Salvare Logo",
               width="200",
               height="200",
            ))),
    html.Hr(),
    html.Div(className="row",
        style={"display":"flex"},
         children=[
             html.Button(id='submit-button-water', n_clicks=0, children='Water'),
             html.Button(id='submit-button-mountain', n_clicks=0, children='Mountain'),
             html.Button(id='submit-button-disaster', n_clicks=0, children='Disaster'),
             html.Button(id='submit-button-all', n_clicks=0, children='Reset')
         ]
        ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        page_current=0,
        sort_mode="single",
        row_selectable="single", 
        selected_rows=[0]       
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
#default graph placeholder, will receive 'figure' from call back
                dcc.Graph(id='graph-id', className='col s12 m6'),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    )
                ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# buttons change the filter for the data
@app.callback(
    Output('datatable-id', 'data'),
    [Input('submit-button-all', 'n_clicks_timestamp'),
    Input('submit-button-water', 'n_clicks_timestamp'),
    Input('submit-button-mountain', 'n_clicks_timestamp'),
    Input('submit-button-disaster', 'n_clicks_timestamp')
    ])
def filter_click(n_click_all, n_click_w, n_click_m, n_click_d): 
    # In Dash 2.4+ there is a better way to determine which button was pushed using callback_context
    # Below is implementation for prior versions
    
    if n_click_all is None:
        n_click_all = 0
    if n_click_w is None:
        n_click_w = 0
    if n_click_m is None:
        n_click_m = 0
    if n_click_d is None:
        n_click_d = 0

    # Water rescue
    if (n_click_w > n_click_m 
        and n_click_w > n_click_d
        and n_click_w > n_click_all):
        return pd.DataFrame.from_records(shelter.read({
            '$and': [
                {
                    '$expr': {
                        '$gte': [
                            {
                                '$toDouble': '$age_upon_outcome_in_weeks'
                            }, 26.0
                        ]
                    }
                }, {
                    '$expr': {
                        '$lte': [
                            {
                                '$toDouble': '$age_upon_outcome_in_weeks'
                            }, 156.0
                        ]
                    }
                }, {
                    'sex_upon_outcome': 'Intact Female'
                }, {
                    'animal_type': 'Dog'
                }, {
                    '$or': [
                        {
                            'breed': 'Labrador Retriever Mix'
                        }, {
                            'breed': re.compile(r"Chesa.*Bay.*Retr.*")
                        }, {
                            'breed': re.compile(r"Newfoundland")
                        }
                    ]
                }
            ]
        })).to_dict('records') 
    
    # Mountain rescue
    if (n_click_m > n_click_w 
        and n_click_m > n_click_d
        and n_click_m > n_click_all):
        return pd.DataFrame.from_records(shelter.read({
            '$and': [
                {
                    '$expr': {
                        '$gte': [
                            {
                                '$toDouble': '$age_upon_outcome_in_weeks'
                            }, 26.0
                        ]
                    }
                }, {
                    '$expr': {
                        '$lte': [
                            {
                                '$toDouble': '$age_upon_outcome_in_weeks'
                            }, 156.0
                        ]
                    }
                }, {
                    'sex_upon_outcome': 'Intact Male'
                }, {
                    'animal_type': 'Dog'
                }, {
                    '$or': [
                        {
                            'breed': 'German Shepherd'
                        }, {
                            'breed': re.compile(r".*Alaskan Malamute.*")
                        }, {
                            'breed': re.compile(r".*Old English Sheepdog.*")
                        }, {
                            'breed': re.compile(r".*Siberian Husky.*")
                        }, {
                            'breed': re.compile(r".*Rottweiler.*")
                        }
                    ]
                }
            ]
        })).to_dict('records') 

    # Disaster rescue
    if (n_click_d > n_click_m
        and n_click_d > n_click_w
        and n_click_d > n_click_all):
        return pd.DataFrame.from_records(shelter.read({
            '$and': [
                {
                    '$expr': {
                        '$gte': [
                            {
                                '$toDouble': '$age_upon_outcome_in_weeks'
                            }, 20.0
                        ]
                    }
                }, {
                    '$expr': {
                        '$lte': [
                            {
                                '$toDouble': '$age_upon_outcome_in_weeks'
                            }, 300.0
                        ]
                    }
                }, {
                    'sex_upon_outcome': 'Intact Male'
                }, {
                    'animal_type': 'Dog'
                }, {
                    '$or': [
                        {
                            'breed': 'German Shepherd'
                        }, {
                            'breed': re.compile(r".*Doberman.*")
                        }, {
                            'breed': re.compile(r".*Golden Retriever.*")
                        }, {
                            'breed': re.compile(r".*Bloodhound.*")
                        }, {
                            'breed': re.compile(r".*Rottweiler.*")
                        }
                    ]
                }
            ]
        })).to_dict('records') 
    
    # All/reset
    return pd.DataFrame.from_records(shelter.read({})).to_dict('records')
    
# change style of selected column
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns is None:
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]
    else:
        return ""

# when data is set, update pie chart
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "data")])
def update_graphs(data):
    dff = pd.DataFrame.from_dict(data)

    breeds_df = dff.groupby(['breed'])['breed'].count().reset_index(name='count')
    shape = breeds_df.shape

    if shape[0] < 20:
        pass
    elif shape[0] < 50:
        breeds_df.loc[breeds_df['count'] < 10, 'breed'] = 'Other breeds'
    elif shape[0] < 100:
        breeds_df.loc[breeds_df['count'] < 50, 'breed'] = 'Other breeds'
    else:
        breeds_df.loc[breeds_df['count'] < 100, 'breed'] = 'Other breeds'
        
    fig = px.pie(breeds_df, values="count", names="breed")

    return fig

# when viewport or selected row changes, update the map    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_viewport_selected_rows")])
def update_map(viewData, selectedRows):
    if not selectedRows is None and not viewData is None:
        viewDF = pd.DataFrame.from_dict(viewData) # convert dict to DataFrame
        dff = viewDF.loc[selectedRows] # get the selected row

        return [
            # center map using coords from selected row
            dl.Map(style={'width': '500px', 'height': '500px'}, center=[dff.iloc[0,13], dff.iloc[0,14]], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[dff.iloc[0,13], dff.iloc[0,14]], children=[
                    dl.Tooltip(dff.iloc[0,4]), # tooltip is breed
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[0,9]), # popup is animal name
                        html.P(dff.iloc[0,4]), # breed
                    ])
                ])
            ])
        ]
    else:
        return []

app